# Packages and Imports

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
!pip install pydicom
!pip install nipype
!pip install dcm2niix
!pip install nibabel
!pip install pycox
!pip install torchtuples
!pip install lifelines


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 2.0 MB 4.4 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 3.2 MB 5.0 MB/s 
     |████████████████████████████████| 5.1 MB 29.5 MB/s 
     |████████████████████████████████| 139 kB 48.0 MB/s 
     |████████████████████████████████| 421 kB 50.7 MB/s 
     |████████████████████████████████| 500 kB 51.2 MB/s 
     |████████████████████████████████| 41 kB 525 kB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 451 kB 4.5 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  Created wheel for dcm2niix: filename=dcm2niix-1.0.20220715-cp38-cp38-linux_x86_64.whl size=601060 sha256=fa2421be966

In [ ]:
import pydicom as di 
import os
from os import listdir
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import torch
from skimage.transform import resize
from subprocess import call
import re
from functools import reduce
import shutil
import nibabel as nib
from PIL import Image
import torch
import torchvision.transforms as transforms

from torch.nn import Module
from torch.nn import Conv2d
from torch.nn import Linear
from torch.nn import MaxPool2d
from torch.nn import ReLU
from torch.nn import LogSoftmax
from torch import flatten

from sklearn.preprocessing import StandardScaler
from sklearn_pandas import DataFrameMapper 
import torchtuples as tt
from pycox.datasets import metabric
from pycox.models import DeepHitSingle
from pycox.evaluation import EvalSurv


import scipy.io



import sys, os, datetime, h5py
import numpy as np
from scipy import io
import pandas as pd
from matplotlib import pyplot
import tensorflow as tf
from tensorflow.keras.models import Model, model_from_json
from tensorflow.keras.layers import Input,InputLayer, Dense,  Dropout, Activation, Concatenate, Lambda
#from tensorflow.python.keras.summary import merge
# from tensorflow.keras.utils import plot_model, multi_gpu_model
from tensorflow.keras.callbacks import CSVLogger, LearningRateScheduler, ModelCheckpoint, EarlyStopping
from tensorflow.keras import backend as K
from tensorflow.keras import regularizers
from functools import partial, update_wrapper
from tensorflow.keras import optimizers

from sklearn.preprocessing import StandardScaler

from lifelines.utils import concordance_index


import nipype.interfaces.fsl as fsl


from nipype.interfaces import fsl
from nipype.testing import example_data

# Metadata

In [ ]:
metadata0 = pd.read_csv('/content/gdrive/MyDrive/dataset/Survival/Year0/healthy_and_ad_patient_0_12_01_2022.csv', sep=',', header=0)
metadata1 = pd.read_csv('/content/gdrive/MyDrive/dataset/Survival/Year1/healthy_and_ad_patient_1_12_01_2022.csv', sep=',', header=0)
metadata2 = pd.read_csv('/content/gdrive/MyDrive/dataset/Survival/Year2/healthy_and_ad_patient_2_12_01_2022.csv', sep=',', header=0)
metadata3 = pd.read_csv('/content/gdrive/MyDrive/dataset/Survival/Year3/healty_and_ad_patient_3_12_01_2022.csv', sep=',', header=0)
metadata4 = pd.read_csv('/content/gdrive/MyDrive/dataset/Survival/Year4/healthy_and_ad_patient_4_12_01_2022.csv', sep=',', header=0)
metadata5 = pd.read_csv('/content/gdrive/MyDrive/dataset/Survival/Year5/heathy_and_ad_patient_5_12_01_2022.csv', sep=',', header=0)

In [ ]:
genetic_data=pd.read_csv('/content/content/ADNIGenetic.csv')

In [ ]:
path='/content/content/ADNI'

In [ ]:
len(os.listdir(path))

327

# Convert DCM Images

In [ ]:
count=0
for dirs in os.listdir(path):
  dirName=os.path.join(path,dirs)
  for v in os.listdir(dirName):
    subDir=os.path.join(dirName,v)
    # print(subDir)
    call(["dcm2niix","-z","y", "-v", "y", "-m", "y" , subDir])
    # for files in os.listdir(subDir):
    #   count+=1
    # if ".gz" in files.lower():
    #   os.remove(os.path.join(dirName, files))



0


# Feature extraction 

In [ ]:
data=[]
for p in covariates:
  features=[]
  for v in os.listdir(os.path.join(path,p)):
    vis=os.path.join(os.path.join(path,p),v)
    for fileName in os.listdir(os.path.join(os.path.join(path,p),v)):
      if '.nii.gz' in fileName.lower():
        tensor=convertDCM2Tensor(os.path.join(vis, fileName))
        feature=feature_extractor(tensor.unsqueeze(0))
        feature=feature.squeeze()
        feature=torch.flatten(feature)
        # print(feature.shape)
        features.append(feature.numpy())
        break
  while len(features) < 5:
    features.append(features[-1])
  data.append(np.array(features).flatten())
  
  
    






# FSL - Grey Matter

In [ ]:
import os

!sudo apt -qq install file
!wget https://fsl.fmrib.ox.ac.uk/fsldownloads/fslinstaller.py
!python2 fslinstaller.py

fslpath = "/usr/local/fsl"
os.environ["FSLDIR"] = fslpath
os.environ["PATH"] += os.pathsep + os.path.join(fslpath, 'bin')
!. ${FSLDIR}/etc/fslconf/fsl.sh

!flirt -version
     


In [ ]:
fslpath = "/content/fsl"
os.environ["FSLDIR"] = fslpath
os.environ["PATH"] += os.pathsep + os.path.join(fslpath, 'bin')
!. ${FSLDIR}/etc/fslconf/fsl.sh

!flirt -version

FLIRT version 6.0


In [ ]:
path='/content/content/ADNI'
count=0
for p in os.listdir(path):
  if ".ipynb" in p:
    continue
  for i,v in enumerate(sorted(os.listdir(os.path.join(path,p)))):
    # print(p,v)
    if len(v) != 1:
      continue
    vis=os.path.join(os.path.join(path,p), v)
    if 'bet.nii' in os.listdir(vis):
      continue
    for fileName in os.listdir(vis):
      if '.nii.gz' in fileName.lower():
        print(os.path.join(vis, fileName))
        btr = fsl.BET()
        btr.inputs.in_file = os.path.join(vis, fileName)
        btr.inputs.out_file = os.path.join(vis, "bet.nii")
        btr.inputs.frac = 0.7
        res = btr.run() 
        count+=1
        break
print(count)


# btr = fsl.BET()
# btr.inputs.in_file = "/content/content/ADNI/003_S_4119/2/2_3_Plane_Localizer_20191029110040_1_i00003.nii.gz"
# btr.inputs.out_file = "/content/content/ADNI/003_S_4119/2/seg3.nii"
# btr.inputs.frac = 0.7
# res = btr.run() 

0


In [ ]:
from nipype.interfaces import fsl
from nipype.testing import example_data

path='/content/content/ADNI'
count=0
for p in os.listdir(path):
  if ".ipynb" in p:
    continue
  for i,v in enumerate(sorted(os.listdir(os.path.join(path,p)))):
    # print(p,v)
    if len(v) != 1:
      continue
    vis=os.path.join(os.path.join(path,p), v)
    if "bet_pve_2.nii" in os.listdir(vis):
      continue
    for fileName in os.listdir(vis):
      if 'bet.nii' ==  fileName.lower():
        print(os.path.join(vis, fileName))
        fastr = fsl.FAST()
        fastr.inputs.in_files = os.path.join(vis, "bet.nii")
# fastr.inputs.out_basename="test2"
        out=fastr.run() 
        count+=1
        break
print(count)



In [ ]:
data=[]

for p in covariates:
  features=[]
  gmv=[]
  for v in os.listdir(os.path.join(path,p)):
    vis=os.path.join(os.path.join(path,p),v)
    for fileName in os.listdir(os.path.join(os.path.join(path,p),v)):
      if 'bet_pve_2.nii' in fileName.lower():

        m=subprocess.run(["fslstats",os.path.join(vis,fileName),'-m'],stdout=subprocess.PIPE)
        m=m.stdout.decode('utf-8').strip()
        v=subprocess.run(["fslstats",os.path.join(vis,fileName),'-v'],stdout=subprocess.PIPE)
        v=v.stdout.decode('utf-8').split()[0].strip()
        # print(m,v)
        mult=float(m)*float(v)
        gmv.append(mult)
        break
  
  gmv=np.array(gmv)
  
  